In [1]:
"""Colab Drive Connection"""

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import warnings
import os 

from collections import defaultdict
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

import networkx as nx
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import fcluster

import random

from tqdm import tqdm_notebook

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

# install datatable
!pip install datatable
import datatable as dt

from numba import njit

import gc

warnings.simplefilter(action="ignore")

# project_home = "/kaggle/input/jane-street-market-prediction"
# data_home = project_home

project_home = "/gdrive/MyDrive/colab/jane-street-market-prediction"
data_home = os.path.join(project_home, "input/data")
model_home = os.path.join(project_home, "output/model")
gs_home = os.path.join(project_home, "output/grid_search")

     |████████████████████████████████| 83.7MB 65kB/s 


https://github.com/FernandoLpz/Optuna-Sklearn-PyTorch/blob/master/optuna_pytorch.py

https://titanwolf.org/Network/Articles/Article?AID=172eb24c-217d-4c4e-9623-e0e68349f4e2#gsc.tab=0

In [3]:
# https://titanwolf.org/Network/Articles/Article?AID=172eb24c-217d-4c4e-9623-e0e68349f4e2#gsc.tab=0
!pip install optuna

     |████████████████████████████████| 296kB 12.2MB/s 
     |████████████████████████████████| 1.1MB 25.8MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 481kB 53.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 143kB 50.2MB/s 
     |████████████████████████████████| 112kB 80.3MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=6fa474fd6db16f63fcdce056389d0b2366c9631f42d6b02e3f79e5dbac2a215c
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5a1b1e2363ed68392d292d215facf588d5448198edd8078bc1
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=2ba523ea244d3742c0e371546cbdeec9794fa91f5b0ddb5bb1129858b8eae3b4
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for pyperclip: filename=pyperclip-1.8.1-cp36-none-any.whl size=11120 sha25

In [4]:
entire_seed = 1029

def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.deterministic = False
    
seed_torch(entire_seed)

In [5]:
train_file = os.path.join(data_home,'train.csv')
features_file = os.path.join(data_home,'features.csv')
example_test_file = os.path.join(data_home,'example_test.csv')
example_sample_submission_file = os.path.join(data_home,'example_sample_submission.csv')

train_data_datatable = dt.fread(train_file)

df_train = train_data_datatable.to_pandas()
df_features = pd.read_csv(features_file)
df_example_test = pd.read_csv(example_test_file)
df_example_sample_submission = pd.read_csv(example_sample_submission_file)

In [6]:
features = [ col for col in df_train.columns if "feature" in col ]
resps = [ col for col in df_train.columns if "resp" in col ]
target_resp = [resp_ for resp_ in resps if "_" not in resp_]
target = ["weight"] + target_resp + features 

In [7]:
"""
Reduce Memory Usage by 75%
https://www.kaggle.com/tomwarrens/nan-values-depending-on-time-of-day
"""

## Reduce Memory

def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
#                 reducing float16 for calculating numpy.nanmean
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

df_train = reduce_memory_usage(df_train)
df_train.info()

Memory usage of dataframe is 2489.4869804382324 MB
Memory usage of dataframe after reduction 1247.0233011245728 MB
Reduced by 49.908422461199 % 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 138 entries, date to ts_id
dtypes: float32(135), int16(1), int32(1), int8(1)
memory usage: 1.2 GB


In [8]:
# drop before 85days
df_train = df_train.loc[df_train.date>85]
# drop weight 0 for training
df_train = df_train.loc[df_train.weight > 0]

# df_labels = df_train[['date','weight','resp_1','resp_2','resp_3','resp_4','resp']]

# df_train = df_train.drop(df_labels.columns,axis=1)

In [9]:
"""
The codes from 'Optimise Speed of Filling-NaN Function'
https://www.kaggle.com/gogo827jz/optimise-speed-of-filling-nan-function
"""

def for_loop(method, matrix, values):
    for i in range(matrix.shape[0]):
        matrix[i] = method(matrix[i], values)
    return matrix

def for_loop_ffill(method, matrix):
    tmp = np.zeros(matrix.shape[1],dtype=np.float32)
    for i in range(matrix.shape[0]):
        matrix[i] = method(matrix[i], tmp)
        tmp = matrix[i]
    return matrix

@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [10]:
# converting numpy for efficient calcualtion.
# ft 1~129
np_ft_train = df_train.loc[:,features[1:]].values
np_ft_train.shape

# ft 0
# np_train_ft0 = df_train.loc[:,features[0]].values

(1571415, 129)

In [11]:
f_mean = np.nanmean(np_ft_train,axis=0)

In [12]:
np_train = df_train.values

In [13]:
print('fillna_npwhere_njit (mean-filling):')
np_train[:,8:-1] = for_loop(fillna_npwhere_njit, np_train[:,8:-1], f_mean)

fillna_npwhere_njit (mean-filling):


In [14]:
dict_features = {col:idx for idx, col in enumerate(df_train.columns.tolist())}

In [15]:
np_d_w = np_train[:,:2]
# ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']
idx_resps = list()
for resp in resps:
    idx_col = dict_features[resp]
    idx_resps.append(idx_col)
np_resps = np_train[:,idx_resps]

In [16]:
resps_prcntls = [50, 49, 49, 50, 50]
resps_prcntls = [np.percentile(np_resps[:,idx], prcntls) for idx, prcntls in enumerate(resps_prcntls)]
resps_prcntls

[2.3540282199974172e-05,
 -2.6968382262566605e-05,
 -6.920687970705338e-05,
 7.239638944156468e-05,
 4.7192643251037225e-05]

In [17]:
list_resps = list()
for idx, resps_prcntl in enumerate(resps_prcntls):
    result = list(map(lambda x: 1 if x > resps_prcntl else 0, np_resps[:,idx]))
    list_resps.append(result)
np_targets = np.stack(list_resps).T

In [18]:
idx_target = [("resp_" not in key) and ("ts_" not in key) for key in dict_features.keys()]
idx_target = np.arange(np_train.shape[1])[idx_target]
X_np_train = np_train[:,idx_target]

In [19]:
X = X_np_train
y = np_targets

In [20]:
from collections import Counter, defaultdict
from sklearn import model_selection

# ---- StratifiedGroupKFold ----
class StratifiedGroupKFold(object):
    """
    StratifiedGroupKFold with random shuffle with a sklearn-like structure
    """

    def __init__(self, n_splits=4, shuffle=True, random_state=42):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, group=None):
        return self.n_splits

    def split(self, X, y, group):
        labels_num = np.max(y) + 1
        y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
        y_distr = Counter()
        # groups = X[group].values
        groups = group
        for label, g in zip(y, groups):
            y_counts_per_group[g][label] += 1
            y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(self.n_splits)])
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)
        
        groups_and_y_counts = list(y_counts_per_group.items())
        random.Random(self.random_state).shuffle(groups_and_y_counts)

        for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(self.n_splits):
                fold_eval = eval_y_counts_per_fold(y_counts, i)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(g)

        all_groups = set(groups)
        for i in range(self.n_splits):
            train_groups = all_groups - groups_per_fold[i]
            test_groups = groups_per_fold[i]

            train_idx = [i for i, g in enumerate(groups) if g in train_groups]
            test_idx = [i for i, g in enumerate(groups) if g in test_groups]

            yield train_idx, test_idx

In [21]:
cv = StratifiedGroupKFold(n_splits=3, random_state=entire_seed)

In [22]:
cv_idxes = [ (train_idx, test_idx) for train_idx, test_idx in cv.split(X, y[:,-1], group=X[:,0])]
for idx, cv_idx in enumerate(cv_idxes):
    train_idx, test_idx = cv_idx
    train_dates = np.unique(X[train_idx, 0]) 
    test_dates = np.unique(X[test_idx, 0])
    print(f"fold {idx+1}"+"*"*30)
    print(train_dates)
    print(test_dates)

fold 1******************************
[ 87.  89.  90.  92.  93.  95.  96.  98. 100. 101. 102. 103. 104. 107.
 109. 110. 111. 113. 116. 117. 118. 119. 120. 122. 124. 126. 127. 129.
 131. 133. 135. 137. 138. 139. 140. 141. 142. 143. 144. 145. 146. 147.
 148. 150. 151. 152. 156. 157. 161. 162. 163. 164. 166. 168. 170. 172.
 173. 174. 175. 177. 181. 183. 184. 185. 188. 193. 195. 196. 197. 203.
 204. 205. 206. 207. 208. 209. 210. 211. 212. 213. 214. 215. 216. 217.
 218. 220. 222. 224. 225. 228. 231. 233. 234. 237. 239. 240. 242. 243.
 246. 247. 248. 250. 251. 252. 253. 254. 256. 257. 260. 262. 263. 264.
 265. 266. 267. 268. 269. 272. 274. 275. 276. 278. 279. 280. 281. 284.
 285. 287. 288. 291. 292. 295. 296. 297. 298. 300. 301. 303. 304. 305.
 306. 307. 308. 310. 311. 313. 318. 320. 321. 323. 324. 325. 327. 329.
 331. 332. 333. 335. 336. 338. 339. 340. 341. 342. 344. 345. 346. 347.
 348. 349. 350. 351. 353. 354. 355. 357. 359. 360. 362. 363. 364. 365.
 367. 368. 369. 370. 371. 373. 374. 375.

In [23]:
class JaneDataset(Dataset):
    def __init__(self, np_X, np_y):
        super(JaneDataset,self).__init__()
        self.X = np_X
        self.y = np_y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        # date, weight, resp
        X_util = self.X[index, :3]
        X = torch.tensor(self.X[index, 3:],dtype=torch.float)
        y = torch.tensor(self.y[index],dtype=torch.float)
        return X_util, X, y

Trial

In [24]:
from optuna.trial import Trial

In [25]:
# # Parameters carry out the trial of category
#   param1  =  Trial . Suggest_categorical ( Name ,  Choices )
# # Perform trial integer parameter
#   param2  =  trial . Suggest_int ( name ,  low ,  high )
# # Perform successive values attempts parameter
#   param3  =  trial . Suggest_uniform ( name ,  low ,  high )
# # Performing trial of discrete values parameters
#   param4  =  trial . Suggest_discrete_uniform ( name ,  low ,  high ,  q )
# # Parameters perform trial logarithm
#   param5  =  trial . Suggest_loguniform ( name ,  low ,  high )

Study

In [26]:
import optuna
# study = optuna.create_study
# study.optimize(objective, n_trials=3)

Hyper parameters for tuning
* Number of convolutional layers (3 to 7)
* Number of filters for each convolutional layer (16, 32, 48, ..., 128)
* Number of units in all bonding layers (100, 200, 300, 400, 500)
* Activation function (ReLU, ELU)
* Optimization method (Adam, MomentumSGD, rmsprop)
* Learning rate (adam_lr (1e-10 ~ 1e-3), momentum_sgd_lr (1e-5 ~ 1e-1))
* weight_decay (1e-10 to 1e-3)

In [27]:
def utility_score(X_d_w,X_r,y):
    # X for date, weight, resp numpy.array
    # y for binary action by random threshold or prediction
    
    # date
#     date_min, date_max = np.min(X_d_w[:,0]), np.max(X_d_w[:,0])
    unq_dates = np.unique(X_d_w[:,0])
    period = len(unq_dates)
#     dates = np.arange(date_min, date_max+1)
    
    list_p = list()
    
    for date in unq_dates:
        idx_date = X_d_w[:,0] == date
        X_d = X_d_w[idx_date,0]
        y_d = y[idx_date]
        w_d = X_d_w[idx_date,1]
        r_d = X_r[idx_date]
        
        p_d = w_d * r_d * y_d
        p = p_d.sum()
        
        list_p.append(p)
    
    np_p = np.array(list_p)
    

    t = np.sum(np_p) / np.sqrt(np.sum(np.power(np_p,2))) * np.sqrt(250/period)
    utility_score = min(max(t,0),6)*np_p.sum()
    return utility_score

In [28]:
class ResnetLinear(nn.Module):
    def __init__(self, trial, num_features, num_classes, df_features, device, verbose=False):
        super(ResnetLinear,self).__init__()

        self.hidden_layer = trial.suggest_categorical("hidden_layer",[512,256,128])
        self.num_layers = trial.suggest_int('n_layers',2,4)
        self.decreasing = trial.suggest_categorical("decreasing",[True, False])
        
        self.f_act = self.get_activation(trial)
        self.dropout = trial.suggest_uniform('dropout',0.1,0.5)
        
        self.embed_dim = trial.suggest_categorical("embed_dim",[0,5,10])

        self.num_features = num_features
        self.num_classes = num_classes
        # self.hidden_layers = hidden_layers
        # self.dropout = dropout
        # self.embed_dim = 0
        self.hidden_layers = None
        self.emb_mode = None

        if verbose:
            print("ResnetLinear Trial")
            print(f"hidden_layer:{self.hidden_layer}; num_layers:{self.num_layers}; decreasing:{self.decreasing}; f_act:{self.f_act}; dropout:{self.dropout}; embed_dim:{self.embed_dim}")        

        if self.embed_dim == 0:
            self.emb_mode = False

        else:
            self.emb_mode = True

            # df_features tag num is 29(fixed value)
            self.n_feat_tags = 29
            # self.embed_dim = selfembed_dim
            self.device = device
            
            self.df_features = df_features.loc[:,df_features.columns[1:]]
            self.df_features["tag_29"] = np.array([1]+[0] * (self.df_features.shape[0]-1))
            self.df_features = self.df_features.astype("int8")
            self.features_tag_matrix = torch.tensor(self.df_features.values).to(self.device)
            
            self.n_feat_tags += 1
            self.tag_embedding = nn.Embedding(self.n_feat_tags+1, self.embed_dim)
            self.tag_weights = nn.Linear(self.n_feat_tags, 1)
            

        self.bn_d0 = nn.BatchNorm1d(self.num_features+ self.embed_dim)
                
        if self.decreasing:
          self.hidden_layers = [int(self.hidden_layer/2**(i)) for i in range(self.num_layers)]
        else:
          self.hidden_layers = [int(self.hidden_layer) for i in range(self.num_layers)]

        self.hidden_layers = [int(self.num_features + self.embed_dim)] + self.hidden_layers

        denses = list()
        
        for i in range(len(self.hidden_layers)-1):
            if i==0:
                denses.append(self.make_layers(self.hidden_layers[i], self.hidden_layers[i+1], self.dropout, self.f_act))
            else:
                denses.append(self.make_layers(self.hidden_layers[i-1]+self.hidden_layers[i], self.hidden_layers[i+1], self.dropout, self.f_act))

        self.denses = nn.Sequential(*denses)
        
        self.out_dense = nn.Linear(self.hidden_layers[-1] + self.hidden_layers[-2], self.num_classes)

    def get_activation(self, trial):
      f_acts = ["ReLU", "SiLU", "LeakyReLU"]
      f_act = trial.suggest_categorical('f_act',f_acts)
      
      if f_act == f_acts[0]: 
          activation = nn.ReLU()
      elif f_act == f_acts[1]: 
          activation = nn.SiLU()
      elif f_act == f_acts[2]: 
          activation = nn.LeakyReLU()
      
      return activation

    def make_layers(self, in_channels, out_channels, dropout=None, f_act=nn.ReLU()):
        layers = list()
        layers.append(nn.Linear(in_channels, out_channels))
        layers.append(nn.BatchNorm1d(out_channels))
        layers.append(f_act)
        
        if dropout:
            layers.append(nn.Dropout(dropout))
        
        module = nn.Sequential(*layers)
        
        return module
    
    # function to make embedding vector of Tag information per Features_0...129
    def features2emb(self):
        # one tag embedding to embed_dim dimension (1,embed_dim) per element
        all_tag_idxs = torch.LongTensor(np.arange(self.n_feat_tags)).to(self.device)
        tag_bools = self.features_tag_matrix
        f_emb = self.tag_embedding(all_tag_idxs).repeat(130,1,1)
        # f_emb에서 tag에 해당하는 값만 f_emb에 남김.
        f_emb = f_emb * tag_bools[:,:,None]
        
        # 각 feature 별로 먗개의 tag가 속하는가?
        s = torch.sum(tag_bools,dim=1)
        # 각 feature 별로 tag값에 해당하여 남겨진 embedding 값을 dimension 별로 합산(1,1,29) / 각 featrue별로 구해진 tag 개수와 division
        f_emb = torch.sum(f_emb, dim=-2) / s[:,None]
        
        return f_emb
        
    def forward(self, x):
        
        # if embedding
        if self.emb_mode:
            f_emb = self.features2emb()
            x = x.view(-1, self.num_features)
            x_emb = torch.matmul(x,f_emb)
            x = torch.hstack((x,x_emb))
        
        # num_features + embed_dim 
        x = self.bn_d0(x)
        
        x_prev = None
        x_now = None

        for idx, dense in enumerate(self.denses):
            if idx == 0:
                x_prev = x
                x_now = dense(x_prev)
                x = torch.cat([x_prev,x_now],1)
                x_prev = x_now
            else:
                x_now = dense(x)
                x = torch.cat([x_prev,x_now],1)
                x_prev = x_now

        x5 = self.out_dense(x)
        
        return x5

In [29]:
class EarlyStopping_GS:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            # print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            # print(f"Validation score improved ({self.val_score:.4f} --> {epoch_score:.4f}). Saving model!")
            self.save_checkpoint(epoch_score, model)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print(f"Validation score improved ({self.val_score:.4f} --> {epoch_score:.4f})")
            # if not DEBUG:
            # torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [30]:
class Model:
    def __init__(self, X, y, cv_idxes, epochs, batch_size, earlystop_num, device, verbose=False):
        self.train_loader = None
        self.valid_loader = None
        
        self.X = X
        self.y = y
        
        self.cv_idxes = cv_idxes
        self.epochs = epochs
        self.batch_size = batch_size
        self.earlystop_num = earlystop_num
        self.device = device 
        self.verbose = verbose

        self.len_cv = len(cv_idxes)

    def prepare_data(self, idx):
        train_idx, valid_idx = cv_idxes[idx]
        X_train = X[train_idx, :]
        y_train = y[train_idx, :]

        X_valid = X[valid_idx, :]
        y_valid = y[valid_idx, :]
        
        train_dataset = JaneDataset(X_train, y_train)
        valid_dataset = JaneDataset(X_valid, y_valid)

        self.train_dataloader = DataLoader(train_dataset, self.batch_size, shuffle=False)
        self.valid_dataloader = DataLoader(valid_dataset, self.batch_size, shuffle=False)

    def get_optimizer(self, trial, model, verbose=False):

        optimizers = ["Adam","SGD"]
        optimizer_name = trial.suggest_categorical('optimizer',optimizers)
        learning_rate = trial.suggest_loguniform('learning_rate',1e-5, 1e-1)
        weight_decay = trial.suggest_loguniform('weight_decay',1e-10,1e-3)

        if optimizer_name == optimizers[0]:
            optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=weight_decay)

        if optimizer_name == optimizers[1]:
            optimizer = optim.SGD(model.parameters(),lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

        if verbose:
            print("Optimzer Trial")          
            print(f"optimizer:{optimizer_name}; learning_rate:{learning_rate}; weight_decay:{weight_decay}")

        return optimizer

    def optimize(self, trial):

        verbosed = None
        if self.verbose:
          verbosed = False

        cv_results = []

        for idx in range(self.len_cv):
            print(f"fold {idx} processing"+"*"*20)
            self.prepare_data(idx)

            if verbosed == False:
                model = ResnetLinear(trial, 130, 5, df_features, self.device, self.verbose)
                optimizer = self.get_optimizer(trial, model, self.verbose)
                verbosed = True

            elif (verbosed == True) | (verbosed is None):
                model = ResnetLinear(trial, 130, 5, df_features, self.device)
                optimizer = self.get_optimizer(trial, model)

            model = model.to(self.device)

            # optimizer = self.get_optimizer(trial, model, self.verbose)
            criterion = nn.BCEWithLogitsLoss()

            es = EarlyStopping_GS(self.earlystop_num, mode="max")

            for epoch in tqdm_notebook(range(self.epochs)):
                running_loss = 0.0
                running_acc = 0.0
                running_auc = 0.0
                running_util = 0.0

                model.train()

                for idx, (X_utils,inputs, labels) in enumerate(self.train_dataloader):

                    optimizer.zero_grad()
                    X_d_w = X_utils[:,:-1].detach().cpu().numpy()
                    X_r = X_utils[:,-1].detach().cpu().numpy()

                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    outputs = model(inputs)
                    
                    true = labels.detach().cpu().numpy()[:,-1]
                    target = np.array(list(map(lambda x: 1 if x > 0.5 else 0, outputs.sigmoid().detach().cpu().numpy()[:,-1])),dtype=np.float)
                    
                    acc = (true == target).sum() / outputs.shape[0]
                    auc = roc_auc_score(true, outputs.detach().cpu().numpy()[:,-1])
                    util = utility_score(X_d_w,X_r,target)
            
                    running_acc += acc
                    running_auc += auc
                    running_util += util

                    loss = criterion(outputs,labels)
                    running_loss += loss.detach().item() * inputs.size(0)
                    loss.backward()
                    optimizer.step()
                    
                epoch_loss = running_loss / len(self.train_dataloader.dataset)
                epoch_acc = running_acc / len(self.train_dataloader)
                epoch_auc = running_auc / len(self.train_dataloader)
                epoch_util = running_util

                model.eval()
                with torch.no_grad():
                    
                    running_loss = 0.0
                    running_acc = 0.0
                    running_auc = 0.0
                    running_util = 0.0

                    for idx, (X_utils, inputs, labels) in enumerate(self.valid_dataloader):

                        X_d_w = X_utils[:,:-1].detach().cpu().numpy()
                        X_r = X_utils[:,-1].detach().cpu().numpy()

                        inputs = inputs.to(device)
                        labels = labels.to(device)

                        outputs = model(inputs)

                        true = labels.detach().cpu().numpy()[:,-1]
                        target = np.array(list(map(lambda x: 1 if x > 0.5 else 0, outputs.sigmoid().detach().cpu().numpy()[:,-1])),dtype=np.float)
                        
                        acc = (true == target).sum() / outputs.shape[0]
                        auc = roc_auc_score(true, outputs.detach().cpu().numpy()[:,-1])
                        util = utility_score(X_d_w,X_r,target)

                        running_acc += acc
                        running_auc += auc
                        running_util += util

                        loss = criterion(outputs, labels)
                        running_loss += loss.detach().item() * inputs.size(0)
                        
                    valid_loss = running_loss / len(self.valid_dataloader.dataset)
                    valid_acc = running_acc / len(self.valid_dataloader)
                    valid_auc = running_auc / len(self.valid_dataloader)
                    valid_util = running_util

                print(f"EPOCH:{epoch+1}|{epochs}; loss(train/valid):{epoch_loss:.4f}/{valid_loss:.4f}; acc(train/valid):{epoch_acc:.4f}/{valid_acc:.4f}; auc(train/valid):{epoch_auc:.4f}/{valid_auc:.4f}; utility(train/valid):{epoch_util:.4f}/{valid_util:.4f}")

                es(valid_util, model)
                if es.early_stop:
                    print("Early stopping")
                    cv_results.append(es.best_score)
                    break

        cv_utility = np.mean(cv_results)
        print(f"mean of cv_results: {cv_utility}")
        return cv_utility

In [31]:
epochs = 100
batch_size = 4096
earlystop_num = 3
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = Model(X, y, cv_idxes, epochs, batch_size, earlystop_num, device, True)

In [34]:
study = optuna.create_study(direction="maximize")

[I 2021-02-20 08:39:10,789] A new study created in memory with name: no-name-9cfc9432-69fb-467e-bfc8-259e0823d3cf


In [ ]:
# https://optuna.readthedocs.io/en/v1.4.0/reference/logging.html
optuna.logging.enable_default_handler()
study.optimize(model.optimize,n_trials=100)

fold 0 processing********************
ResnetLinear Trial
hidden_layer:256; num_layers:4; decreasing:False; f_act:LeakyReLU(negative_slope=0.01); dropout:0.3713539211781073; embed_dim:5
Optimzer Trial
optimizer:SGD; learning_rate:0.0060669990085953545; weight_decay:3.4870532320747824e-08


In [ ]:
# gs_home = os.path.join(project_home, "output/grid_search")
file_name = "opt_reslin.pkl"
file_path = os.path.join(gs_home, file_name)

df_optuna_result = study.trials_dataframe()
df_optuna_result.to_pickle(file_path)